In [2]:
import os
import shutil
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from threading import Thread
from PIL import Image

In [3]:
import glob
import pandas as pd
import xml.etree.ElementTree as ET

**Definição da classe pra gerar imagens com ImageDataGenerator**

In [9]:
# teste de cada alteração na imagem.
class GenerateImage(Thread):
    
    def __init__ (self, path, to, qtd):
        Thread.__init__(self)
        self.path = path
        self.to = to
        self.qtd = qtd
        
    def run(self):
        self.generate()
        
    """
    To run this script do the following:
    1. Ensure you open the command prompt in the folder that contains
    train, test and validation (optional).
    2. If you just want to work with train and test, remove validation in line 39.
    3. Run python3 xmltocsv.py
    """

    def rename_xml(self,xml_file,image_file):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        
        root.find('filename').text = os.path.split(os.path.abspath(image_file))[1] #nome da imagem com tipo
        root.find('path').text = "" #retira qualquer valor que tenha nesse atributo.
        root.find('folder').text = ""
        tree.write(xml_file)
                
    def generate(self):

        uri_images = glob.glob(self.path + '/*.jpg')#lista de nomes de arquivos que existem na pasta path
        #objeto de ImageDataGenerator, que gera imagens
        datagenerator = ImageDataGenerator(
            rotation_range = 20,
            width_shift_range=0.1,
            height_shift_range=0.1,
            #rescale=1./255,
            shear_range=0.2,
            zoom_range=0.3,
            horizontal_flip=True,
            #vertical_flip=True,
            fill_mode='nearest'
        )

        #calculo da quantidade a ser repetida para cada imagem encontrada
        count = int(self.qtd/len(uri_images))
        count = count if (count > 0) else 1
        
        qi = 0
            
        #criar imagens a partir da lista na pasta
        for uri_image in uri_images:
            if not os.path.isfile(uri_image): #Se não for arquivo, pula esse
                continue
                
            
            #cria uma pasta correspondente para essa imagem
            try:
                os.mkdir(self.to+"/"+os.path.split(os.path.abspath(uri_image))[1].replace(".jpg",""))
            except:
                pass
            
            uri_image_name = uri_image.replace(".jpg", "") #a uri da imagem sem o tipo
            name_image_without_type = os.path.split(os.path.abspath(uri_image))[1].replace(".jpg","") #apenas o nome da imagem
            
            image = load_img(uri_image) # carrega a imagem
            image_array = img_to_array(image)
            image_array = image_array.reshape((1,)+image_array.shape) #redimensiona a imagem

            i = 0
            for batch in datagenerator.flow(image_array, 
                                            batch_size=1, 
                                            save_to_dir=self.to+"/"+name_image_without_type, 
                                            save_prefix="g", 
                                            save_format='jpg'):
                
                shutil.copy(uri_image_name+".xml",self.to+"/"+name_image_without_type+"/"+str(i)+".xml") #copia o arquivo xml para a pasta correspondente da imagem.
                
                print('.', end="")
                i = i+1
                if (i >= count):
                    break
            
            #dentro desta pasta: a pasta correspondente ao aumento de dados da imagem, renomeia os xml para cada imagem
            self.rename_all_xml(self.to+"/"+name_image_without_type)
            
            qi = i+1
            if qi >= self.qtd:
                break
            
            
    def rename_all_xml(self, path_image):
        uri_images = glob.glob(path_image+'/*.jpg')
        uri_xmls = glob.glob(path_image+'/*.xml')
        
        i = 0
        for uri_image in uri_images:
            uri_name = uri_image.replace(".jpg", "")
            
            #se o arquivo já existir
            if os.path.isfile(uri_name+".xml"):
                os.remove(uri_xmls[i])
                continue

            os.rename(uri_xmls[i], uri_name+".xml")
            
            self.rename_xml(uri_name+".xml", uri_image)
            
            i = i+1
        

**redimensionar as imagens para Uma resolução desejada**

In [47]:
#redimensiona cada imagem da path para o tamanho correspondente
class CropScalingImage(Thread):
    
    def __init__ (self, path, to_path, width, height, with_crop, with_resize):
        Thread.__init__(self)
        self.path = path
        self.to_path = to_path
        self.width = width
        self.height = height
        self.with_crop = with_crop
        self.with_resize = with_resize
        
    def run(self):
        self.scaling()
        
    def scaling(self):
        
        for name in os.listdir(self.path):
        
            img = Image.open(self.path+"/"+name)

            #recortar a imagem
            
            w = img.size[0]
            h = img.size[1]
            
            if self.with_crop:
                if w > h:
                    dw = (w-h)/2.0 #position 0 is width, 1 is height
                    img = img.crop((dw,0,w-dw,h))
                elif h > w:
                    dh = (h-w)/2.0 #position 0 is width, 1 is height
                    img = img.crop((0,dh,w,h-dh))
            
            if self.with_resize:
                #redimensionar a imagem
                img = img.resize((self.width,self.height), Image.ANTIALIAS)
                
            img.save(self.to_path+"/"+name)
            
            print(".", end="")

In [13]:
class GrayScale(Thread):
    
    def __init__ (self, path):
        Thread.__init__(self)
        self.path = path
        
    def run(self):
        self.doIt()
        
    def doIt(self):        
        images = os.listdir(self.path)
            
        for image in images:
            img = Image.open(self.path+"/"+image)

            # converte a imagem para o modo L (escala de cinza)
            img = img.convert('L')

            # salva a nova imagem
            img.save(self.path+"/"+image)

# Executar

In [23]:
## setar todos arquivos xml para nome específico
def set_xml(path):

    for sub_path in os.listdir(path):
        for uri_xml in glob.glob(path+"/"+sub_path+'/*.xml'):
            print(uri_xml+" ", end="")
            #file_name = image_name
            tree = ET.parse(uri_xml)
            root = tree.getroot()

            uri = root.find('filename').text
            name = os.path.split(os.path.abspath(uri))[1] #com tipo
            root.find('filename').text = name
            root.find('path').text = ""
            root.find('folder').text = ""

            print(name)
            tree.write(uri_xml)

In [24]:
for path in os.listdir("C:\\Users\\Joelp\\OneDrive\\Imagens\\blocos_ufrb\\Gerada"):
    print(path+" ")
    set_xml("C:\\Users\\Joelp\\OneDrive\\Imagens\\blocos_ufrb\\Gerada\\"+path)

ANTIGA_BIBLIOTECA 
C:\Users\Joelp\OneDrive\Imagens\blocos_ufrb\Gerada\ANTIGA_BIBLIOTECA/0\g_0_6674.xml g_0_6674.jpg
C:\Users\Joelp\OneDrive\Imagens\blocos_ufrb\Gerada\ANTIGA_BIBLIOTECA/1\g_0_9763.xml g_0_9763.jpg
C:\Users\Joelp\OneDrive\Imagens\blocos_ufrb\Gerada\ANTIGA_BIBLIOTECA/10\g_0_446.xml g_0_446.jpg
C:\Users\Joelp\OneDrive\Imagens\blocos_ufrb\Gerada\ANTIGA_BIBLIOTECA/11\g_0_7565.xml g_0_7565.jpg
C:\Users\Joelp\OneDrive\Imagens\blocos_ufrb\Gerada\ANTIGA_BIBLIOTECA/12\g_0_9798.xml g_0_9798.jpg
C:\Users\Joelp\OneDrive\Imagens\blocos_ufrb\Gerada\ANTIGA_BIBLIOTECA/13\g_0_6536.xml g_0_6536.jpg
C:\Users\Joelp\OneDrive\Imagens\blocos_ufrb\Gerada\ANTIGA_BIBLIOTECA/14\g_0_2810.xml g_0_2810.jpg
C:\Users\Joelp\OneDrive\Imagens\blocos_ufrb\Gerada\ANTIGA_BIBLIOTECA/15\g_0_2114.xml g_0_2114.jpg
C:\Users\Joelp\OneDrive\Imagens\blocos_ufrb\Gerada\ANTIGA_BIBLIOTECA/16\g_0_3572.xml g_0_3572.jpg
C:\Users\Joelp\OneDrive\Imagens\blocos_ufrb\Gerada\ANTIGA_BIBLIOTECA/17\g_0_8847.xml g_0_8847.jpg
C:\Us

C:\Users\Joelp\OneDrive\Imagens\blocos_ufrb\Gerada\PAV_2/1\g_0_8134.xml g_0_8134.jpg
C:\Users\Joelp\OneDrive\Imagens\blocos_ufrb\Gerada\PAV_2/10\g_0_5941.xml g_0_5941.jpg
C:\Users\Joelp\OneDrive\Imagens\blocos_ufrb\Gerada\PAV_2/11\g_0_8104.xml g_0_8104.jpg
C:\Users\Joelp\OneDrive\Imagens\blocos_ufrb\Gerada\PAV_2/12\g_0_1884.xml g_0_1884.jpg
C:\Users\Joelp\OneDrive\Imagens\blocos_ufrb\Gerada\PAV_2/13\g_0_825.xml g_0_825.jpg
C:\Users\Joelp\OneDrive\Imagens\blocos_ufrb\Gerada\PAV_2/14\g_0_4709.xml g_0_4709.jpg
C:\Users\Joelp\OneDrive\Imagens\blocos_ufrb\Gerada\PAV_2/15\g_0_8659.xml g_0_8659.jpg
C:\Users\Joelp\OneDrive\Imagens\blocos_ufrb\Gerada\PAV_2/16\g_0_1359.xml g_0_1359.jpg
C:\Users\Joelp\OneDrive\Imagens\blocos_ufrb\Gerada\PAV_2/17\g_0_9960.xml g_0_9960.jpg
C:\Users\Joelp\OneDrive\Imagens\blocos_ufrb\Gerada\PAV_2/18\g_0_4267.xml g_0_4267.jpg
C:\Users\Joelp\OneDrive\Imagens\blocos_ufrb\Gerada\PAV_2/19\g_0_6178.xml g_0_6178.jpg
C:\Users\Joelp\OneDrive\Imagens\blocos_ufrb\Gerada\PAV_2/

C:\Users\Joelp\OneDrive\Imagens\blocos_ufrb\Gerada\PREDIO_SOLOS/25\g_0_3144.xml g_0_3144.jpg
C:\Users\Joelp\OneDrive\Imagens\blocos_ufrb\Gerada\PREDIO_SOLOS/26\g_0_4909.xml g_0_4909.jpg
C:\Users\Joelp\OneDrive\Imagens\blocos_ufrb\Gerada\PREDIO_SOLOS/27\g_0_5781.xml g_0_5781.jpg
C:\Users\Joelp\OneDrive\Imagens\blocos_ufrb\Gerada\PREDIO_SOLOS/28\g_0_8207.xml g_0_8207.jpg
C:\Users\Joelp\OneDrive\Imagens\blocos_ufrb\Gerada\PREDIO_SOLOS/29\g_0_8390.xml g_0_8390.jpg
C:\Users\Joelp\OneDrive\Imagens\blocos_ufrb\Gerada\PREDIO_SOLOS/3\g_0_2236.xml g_0_2236.jpg
C:\Users\Joelp\OneDrive\Imagens\blocos_ufrb\Gerada\PREDIO_SOLOS/30\g_0_3556.xml g_0_3556.jpg
C:\Users\Joelp\OneDrive\Imagens\blocos_ufrb\Gerada\PREDIO_SOLOS/31\g_0_7613.xml g_0_7613.jpg
C:\Users\Joelp\OneDrive\Imagens\blocos_ufrb\Gerada\PREDIO_SOLOS/32\g_0_4639.xml g_0_4639.jpg
C:\Users\Joelp\OneDrive\Imagens\blocos_ufrb\Gerada\PREDIO_SOLOS/33\g_0_3557.xml g_0_3557.jpg
C:\Users\Joelp\OneDrive\Imagens\blocos_ufrb\Gerada\PREDIO_SOLOS/34\g_0_

In [12]:
def rename_xml(xml_file,image_file):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        
        root.find('filename').text = image_file #nome da imagem com tipo
        root.find('path').text = "" #retira qualquer valor que tenha nesse atributo.
        root.find('folder').text = ""
        tree.write(xml_file)

In [16]:
def move_to_unique_folder(path,to):
    
    for class_ in os.listdir(path):
        for folder in os.listdir(path+"/"+class_):

            for file in glob.glob(path+"/"+class_+"/"+folder+'/*.xml'):
                q_to = len(glob.glob(to+"/*.xml"))
                name_file = os.path.split(os.path.abspath(file))[1].replace(".xml", "")
                    
                name_file1 = "g_"+str(q_to)
                uri_xml = to+"/"+name_file1+".xml"
                uri_image = to+"/"+name_file1+".jpg"
                    
                try:
                    os.rename(file,uri_xml) #move o xml para a pasta correspondente
                    rename_xml(uri_xml,name_file1+".jpg") #renomeia o nome da imagem no xml
                    os.rename(path+"/"+class_+"/"+folder+"/"+name_file+".jpg", uri_image) #move a imagem
                except:
                    continue

                #os.remove(path+"/"+class_+"/"+folder+"/"+name_file+".jpg") #remove  a imagem
                #os.remove(file) #remove o arquivo
                print(".", end="")
            
    print("complete.")

In [17]:
#move e renomeia as imagens para uma pasta
move_to_unique_folder("C:/Users/Joelp/OneDrive/Imagens/blocos_ufrb/Gerada",
                      "C:/Users/Joelp/OneDrive/Imagens/blocos_ufrb/Train")

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [48]:
#recorte e redimensionamento
classes = os.listdir("C:/Users/Joelp/OneDrive/Imagens/blocos_ufrb/Original")

for classe in classes:
    c_s1 = CropScalingImage("C:/Users/Joelp/OneDrive/Imagens/blocos_ufrb/Original/"+classe, "C:/Users/Joelp/OneDrive/Imagens/blocos_ufrb/Editada/"+classe, 930, 930, True, True)
    c_s1.start()

................................................................................................................................................................................................................................................................................................................................

In [45]:
#escala de cinza
classes = os.listdir("C:/Users/Joelp/OneDrive/Imagens/blocos_ufrb/Editadas")

for classe in classes:
    sc = GrayScale("ufrb/validation")
    sc.start()

FileNotFoundError: [WinError 3] O sistema não pode encontrar o caminho especificado: 'C:/Users/Joelp/OneDrive/Imagens/blocos_ufrb/Editadas'

**Balancear as quantidades de imagens**

In [10]:
#geração de imagens

classes = os.listdir("C:/Users/Joelp/OneDrive/Imagens/blocos_ufrb/Editada")
for classe in classes:
    g = GenerateImage("C:/Users/Joelp/OneDrive/Imagens/blocos_ufrb/Editada/"+classe, "C:/Users/Joelp/OneDrive/Imagens/blocos_ufrb/Gerada/"+classe,2000)
    g.start()

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

**Apagar imagens em excesso**

In [13]:
classes = os.listdir("imagens/sementes/deep_learning")

for classe in classes:
    print(classe)
    print(len(os.listdir("imagens/sementes/deep_learning/"+classe)))

abobora
4245
milho
4216


In [18]:
#
i = 1
images = os.listdir("imagens/sementes/deep_learning/milho")
for image in images:
    os.remove("imagens/sementes/deep_learning/milho/"+image)
    print(".", end="")
    if i == 1316:
        break
    i = i+1

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

**Converter imagens png para jpeg**

In [5]:
list_classes = ["castanha","feijao_carioquinha","jaca","limao","mamao"]

for classe in list_classes:
    images = os.listdir("sementes/deep_learning/"+classe)
    print(classe)
    for image in images:
        im = Image.open("sementes/deep_learning/"+classe+"/"+image)
        rgb_im = im.convert('RGB')
        rgb_im.save("sementes/deep_learning/"+classe+"/"+image)
        print(".", end="")

**Renomear as imagens**

In [7]:
list_classes = ["castanha","feijao_carioquinha","jaca","limao","mamao"]
for classe in list_classes:
    list_sementes = os.listdir("sementes/deep_learning/"+classe)
    i = 0
    print("\n"+classe)
    for name in list_sementes:
        
        os.rename("sementes/deep_learning/"+classe+"/"+name,"sementes/deep_learning/"+classe+"/"+classe+"_"+str(i)+".jpeg")
        i = i+1
        print(".",end="")

castanha
...............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................